In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')

In [ ]:
display(train_df.head())
display(train_df.info())

In [ ]:
print('We have', len(train_df.Country_Region.unique()), 'countries/regions in the dataset.')
print('We have', len(train_df.Province_State.unique()), 'provinces/states in the dataset.')

In [ ]:
num_fatalities = train_df.loc[train_df['Date'] == train_df['Date'].max()]['Fatalities'].sum()
print('Between {} and {} there are {} fatalities.'.format(train_df['Date'].min(), train_df['Date'].max(), int(num_fatalities)))

In [ ]:
cz_sick = int(train_df.loc[(train_df['Country_Region'] == 'Czechia') & (train_df['Date'] == train_df['Date'].max())]['ConfirmedCases'].values[0])
cz_fatalities = int(train_df.loc[(train_df['Country_Region'] == 'Czechia') & (train_df['Date'] == train_df['Date'].max())]['Fatalities'].values[0])
print('In Czech Republic there are {} confirmed cases and {} fatalities.'.format(cz_sick, cz_fatalities))

In [ ]:
train_df['UniqueRegion'] = np.where(train_df['Province_State'].isna(), train_df['Country_Region'], train_df['Country_Region'] + ' - ' + train_df['Province_State'])
train_df.loc[train_df['Country_Region'] == 'China'].head()

In [ ]:
# Calculate sick per day

countries = train_df['UniqueRegion'].unique()
train_df['SickPerDay'] = 0

for country in countries:
    len_country = len(train_df.loc[train_df['UniqueRegion'] == country])
    len_diffs = len(train_df.loc[train_df['UniqueRegion'] == country]['ConfirmedCases'].diff())
    if len_country > 65 or len_diffs > 65:
        raise NameError('Too many rows for country {}'.format(country))
    train_df['SickPerDay'].loc[(train_df['UniqueRegion'] == country)] = train_df.loc[train_df['UniqueRegion'] == country]['ConfirmedCases'].diff()
    

train_df['SickPerDay'] = train_df['SickPerDay'].fillna(0)
display(train_df.loc[train_df['UniqueRegion'] == 'Czechia'].tail())
display(train_df.info())

In [ ]:
# Draw timetrend for ten most affected countries
# Draw timetrend for ten least affected countries

# Span from first sick to first deceased
# Span from first sick to first 100 sick

# Mortality rate per country

In [ ]:
timetrend_sick = sns.lineplot(train_df['Date'], train_df['ConfirmedCases'])

In [ ]:
timetrend_deceased = sns.lineplot(train_df['Date'], train_df['Fatalities'])

In [ ]:
sns.lineplot(train_df.loc[(train_df['UniqueRegion'] == 'Czechia') & (train_df['ConfirmedCases'] > 0)]['Date'], train_df.loc[(train_df['UniqueRegion'] == 'Czechia')& (train_df['ConfirmedCases'] > 0)]['ConfirmedCases'])

In [ ]:
sns.lineplot(x=train_df.loc[(train_df['UniqueRegion'] == 'Czechia') & (train_df['SickPerDay'] > 0)]['Date'],
             y=train_df.loc[(train_df['UniqueRegion'] == 'Czechia')& (train_df['SickPerDay'] > 0)]['SickPerDay'])

In [ ]:
top10_most_cases = train_df.loc[train_df['Date'] == train_df['Date'].max()][['UniqueRegion','ConfirmedCases']].sort_values(by='ConfirmedCases', ascending=False).head(10)
top10_most_deceased = train_df.loc[train_df['Date'] == train_df['Date'].max()][['UniqueRegion','Fatalities']].sort_values(by='Fatalities', ascending=False).head(10)
top10_most_sick_per_day = train_df.loc[train_df['Date'] == train_df['Date'].max()][['UniqueRegion','SickPerDay']].sort_values(by='SickPerDay', ascending=False).head(10)

In [ ]:
top10_most_cases_df = train_df.loc[train_df['UniqueRegion'].isin(top10_most_cases['UniqueRegion'].values)]

In [ ]:
# Transform the dataframe to show each country in a different column

main_df = pd.DataFrame()

for i, top10_country in enumerate (top10_most_cases_df['UniqueRegion'].unique()):
    if i == 0:
        main_df = top10_most_cases_df.loc[top10_most_cases_df['UniqueRegion'] == top10_country][['Date', 'ConfirmedCases']].sort_values(by='Date')
        main_df = main_df.rename({'ConfirmedCases': top10_country}, axis='columns')

    else:
        temp_df = top10_most_cases_df.loc[top10_most_cases_df['UniqueRegion'] == top10_country][['Date', 'ConfirmedCases']]
        temp_df = temp_df.rename({'ConfirmedCases': top10_country}, axis='columns')
        main_df = pd.merge(main_df, temp_df, on=['Date'])

main_df = main_df.set_index('Date')
main_df.head()

In [ ]:
main_df.plot(figsize=(20,10))

In [ ]:
# Build an LSTM first for one country only (Czechia)
# Then expand into all countries
# Figure out how to do predictions on the test set
# Submit predictions
# Expand on the model, add more features, external datasets etc.